In [1]:
from langchain_ollama import OllamaLLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
import numpy as np

In [6]:
import pandas as pd

In [2]:
model = OllamaLLM(model="llama3")

In [3]:
result = model.invoke(input="This is your instruction: You'll be given a task of assessing whether a syllogism is valid or invalid. For one message I would like to hear only one word answer. Firstly, you'll read two premises and after that: proposed conclusion. Okay?")

In [4]:
result

'Ready!'

# Animal Round

In [5]:
import pandas as pd

excel_df = pd.read_excel("Próba_Dla_Maszyny.xlsx")
# excel_df["Text"][0]

len(excel_df)

2052

In [7]:
results = []
for i in range(len(excel_df)):

        result = model.invoke(f"""
                {excel_df["Syllogism"][i]}
                        
                Is this conclusion valid or invalid? Answer with one word:""")
        
        results.append(result)

excel_df["Llama3 Results"] = results

In [9]:
excel_df.to_excel("results.xlsx")

In [17]:
excel_df

,Unnamed: 0,Syllogism,Content Theme,Content Validity,Mood,Figure,Syllogism Result,Llama3 Results
0,44,Some Humans are Mammals. All Mammals are Anima...,Animals,0,IAI,1,0,Invalid.
1,1805,No Squirrels are Rodents. All Rodents are Anim...,Animals,0,EAA,1,0,Invalid.
2,1068,Some Humans are Mammals. All Mammals are Anima...,Animals,0,IAA,1,0,Valid.
3,860,No Lemurs are Primates. All Primates are Anima...,Animals,0,EAA,1,0,Valid.
4,1051,Some Whales are Mammals. All Mammals are Anima...,Animals,0,IAA,1,0,Invalid
...,...,...,...,...,...,...,...,...
2047,652,No Tigers are Carnivores. Some Carnivores are ...,Animals,0,EIE,1,0,Invalid
2048,975,No Earthworms are Worms. No Worms are Animals....,Animals,0,EEA,1,0,Valid.
2049,952,Some Earthworms are Worms. All Worms are Anima...,Animals,0,IAA,1,0,Valid.
2050,1478,All Beetles are Insects. All Insects are Anima...,Animals,0,AAE,1,0,Invalid


In [33]:
excel_df["Llama3 Results"].value_counts()

Llama3 Results
Invalid     945
Invalid.    508
Valid       363
Valid.      236
Name: count, dtype: int64

In [34]:
mask = (excel_df['Syllogism Result'] == 0) & (excel_df['Llama3 Results'].isin(['Invalid', 'Invalid.']))
excel_df["True_Negative"] = mask

mask = (excel_df['Syllogism Result'] == 1) & (excel_df['Llama3 Results'].isin(['Invalid', 'Invalid.']))
excel_df["False_Negative"] = mask

mask = (excel_df['Syllogism Result'] == 1) & (excel_df['Llama3 Results'].isin(['Valid', 'Valid.']))
excel_df["True_Positive"] = mask

mask = (excel_df['Syllogism Result'] == 0) & (excel_df['Llama3 Results'].isin(['Valid', 'Valid.']))
excel_df["False_Positive"] = mask

In [80]:
excel_df['Llama3 Results'] = excel_df['Llama3 Results'].str.replace('.', '', regex=False)

valid_streak = (excel_df['Llama3 Results'] == "Valid") & (excel_df['Llama3 Results'].shift(-1) == "Valid")
invalid_streak = (excel_df['Llama3 Results'] == "Invalid") & (excel_df['Llama3 Results'].shift(-1) == "Invalid")

correct_streak = ((excel_df["True_Positive"] == 1) | (excel_df["True_Negative"] == 1)) & ((excel_df["True_Positive"].shift() == 1) | (excel_df["True_Negative"].shift() == 1))

excel_df["Valid Streak"] = valid_streak
excel_df["Invalid Streak"] = invalid_streak
excel_df["Correct Streak"] = correct_streak



In [83]:
excel_df= excel_df.astype({'True_Negative': 'int', 'False_Negative': 'int', "True_Positive": 'int', "False_Positive": 'int', "Valid Streak": "int", "Invalid Streak": "int", "Correct Streak": "int"})

In [84]:
excel_df

,Unnamed: 0,Syllogism,Content Theme,Content Validity,Mood,Figure,Syllogism Result,Llama3 Results,True_Negative,False_Negative,True_Positive,False_Positive,Valid Streak,Invalid Streak,Correct Streak
0,44,Some Humans are Mammals. All Mammals are Anima...,Animals,0,IAI,1,0,Invalid,1,0,0,0,0,1,0
1,1805,No Squirrels are Rodents. All Rodents are Anim...,Animals,0,EAA,1,0,Invalid,1,0,0,0,0,0,1
2,1068,Some Humans are Mammals. All Mammals are Anima...,Animals,0,IAA,1,0,Valid,0,0,0,1,1,0,0
3,860,No Lemurs are Primates. All Primates are Anima...,Animals,0,EAA,1,0,Valid,0,0,0,1,0,0,0
4,1051,Some Whales are Mammals. All Mammals are Anima...,Animals,0,IAA,1,0,Invalid,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,652,No Tigers are Carnivores. Some Carnivores are ...,Animals,0,EIE,1,0,Invalid,1,0,0,0,0,0,0
2048,975,No Earthworms are Worms. No Worms are Animals....,Animals,0,EEA,1,0,Valid,0,0,0,1,1,0,0
2049,952,Some Earthworms are Worms. All Worms are Anima...,Animals,0,IAA,1,0,Valid,0,0,0,1,0,0,0
2050,1478,All Beetles are Insects. All Insects are Anima...,Animals,0,AAE,1,0,Invalid,1,0,0,0,0,1,0


In [98]:
false_positive = excel_df["False_Positive"].sum() 
true_positive = excel_df["True_Positive"].sum()
false_negative = excel_df["False_Negative"].sum()
true_negative = excel_df["True_Negative"].sum()

# Real Positives: {(excel_df["Syllogism Result"] == 1).sum()}

accuracy = (true_negative + true_positive) / len(excel_df)
print(f"Accuracy: {accuracy}")
precision = true_positive / (true_positive + false_positive)
print(f"Precision: {precision}")

recall = true_positive / (true_positive + false_negative)
print(f"Recall: {recall}")

print(f"False Positive: {false_positive}, False Positive Rate: {false_positive / (false_positive + true_negative)}")
print(f"True Positive: {true_positive}, True Positive Rate (Sensitivity): {true_positive / (true_positive + false_negative)}")
print(f"False Negative: {false_negative}, False Negative Rate (Miss Rate): {false_negative / (false_negative + true_positive)}")
print(f"True Negative: {true_negative}, True Negative Rate: {true_negative / (true_negative + false_negative)}")

Accuracy: 0.7017543859649122
Precision: 0.16026711185308848
Recall: 0.4682926829268293
False Positive: 503, False Positive Rate: 0.27233351380617216
True Positive: 96, True Positive Rate (Sensitivity): 0.4682926829268293
False Negative: 109, False Negative Rate (Miss Rate): 0.5317073170731708
True Negative: 1344, True Negative Rate: 0.9249827942188575


In [97]:
excel_df.to_excel("llama_results.xlsx")

# Nonsense Round

In [7]:
excel_df = pd.read_excel("Próba_Dla_Maszyny_Wielka_Abstrakcja_.xlsx")
# excel_df["Text"][0]

len(excel_df)

results = []
for i in range(len(excel_df)):

        result = model.invoke(f"""
                {excel_df["Syllogism"][i]}
                        
                Is this conclusion valid or invalid? Answer with one word:""")
        
        results.append(result)

excel_df["Llama3 Results"] = results

In [8]:
excel_df["Llama3 Results"].value_counts()

Llama3 Results
Invalid     423
Invalid.    250
Valid       192
Valid.      161
Name: count, dtype: int64

In [9]:
mask = (excel_df['Syllogism Result'] == 0) & (excel_df['Llama3 Results'].isin(['Invalid', 'Invalid.']))
excel_df["True_Negative"] = mask

mask = (excel_df['Syllogism Result'] == 1) & (excel_df['Llama3 Results'].isin(['Invalid', 'Invalid.']))
excel_df["False_Negative"] = mask

mask = (excel_df['Syllogism Result'] == 1) & (excel_df['Llama3 Results'].isin(['Valid', 'Valid.']))
excel_df["True_Positive"] = mask

mask = (excel_df['Syllogism Result'] == 0) & (excel_df['Llama3 Results'].isin(['Valid', 'Valid.']))
excel_df["False_Positive"] = mask

In [10]:
false_positive = excel_df["False_Positive"].sum() 
true_positive = excel_df["True_Positive"].sum()
false_negative = excel_df["False_Negative"].sum()
true_negative = excel_df["True_Negative"].sum()

# Real Positives: {(excel_df["Syllogism Result"] == 1).sum()}

accuracy = (true_negative + true_positive) / len(excel_df)
print(f"Accuracy: {accuracy}")
precision = true_positive / (true_positive + false_positive)
print(f"Precision: {precision}")

recall = true_positive / (true_positive + false_negative)
print(f"Recall: {recall}")

print(f"False Positive: {false_positive}, False Positive Rate: {false_positive / (false_positive + true_negative)}")
print(f"True Positive: {true_positive}, True Positive Rate (Sensitivity): {true_positive / (true_positive + false_negative)}")
print(f"False Negative: {false_negative}, False Negative Rate (Miss Rate): {false_negative / (false_negative + true_positive)}")
print(f"True Negative: {true_negative}, True Negative Rate: {true_negative / (true_negative + false_negative)}")

Accuracy: 0.6968810916179338
Precision: 0.22662889518413598
Recall: 0.6779661016949152
False Positive: 273, False Positive Rate: 0.30066079295154186
True Positive: 80, True Positive Rate (Sensitivity): 0.6779661016949152
False Negative: 38, False Negative Rate (Miss Rate): 0.3220338983050847
True Negative: 635, True Negative Rate: 0.9435364041604755


In [11]:
excel_df['Llama3 Results'] = excel_df['Llama3 Results'].str.replace('.', '', regex=False)

valid_streak = (excel_df['Llama3 Results'] == "Valid") & (excel_df['Llama3 Results'].shift(-1) == "Valid")
invalid_streak = (excel_df['Llama3 Results'] == "Invalid") & (excel_df['Llama3 Results'].shift(-1) == "Invalid")

correct_streak = ((excel_df["True_Positive"] == 1) | (excel_df["True_Negative"] == 1)) & ((excel_df["True_Positive"].shift() == 1) | (excel_df["True_Negative"].shift() == 1))

excel_df["Valid Streak"] = valid_streak
excel_df["Invalid Streak"] = invalid_streak
excel_df["Correct Streak"] = correct_streak

In [12]:
excel_df= excel_df.astype({'True_Negative': 'int', 'False_Negative': 'int', "True_Positive": 'int', "False_Positive": 'int', "Valid Streak": "int", "Invalid Streak": "int", "Correct Streak": "int"})

In [13]:
excel_df

,Unnamed: 0,Syllogism,Content Theme,Content Validity,Mood,Figure,Syllogism Result,Llama3 Results,True_Negative,False_Negative,True_Positive,False_Positive,Valid Streak,Invalid Streak,Correct Streak
0,322,Some lScwArKxAT are jIzWPDwUUk. Some jIzWPDwUU...,Nonsense,0,III,1,0,Valid,0,0,0,1,0,0,0
1,739,All iUyshCysyl are mRTPduzuwG. No mRTPduzuwG a...,Nonsense,0,AEI,1,0,Invalid,1,0,0,0,0,0,0
2,693,All HATdmaWUvU are PFlRHAaJIU. Some PFlRHAaJIU...,Nonsense,1,AII,1,1,Valid,0,0,1,0,1,0,1
3,49,No ssVZCGyCJw are LwpSUcgkmz. All LwpSUcgkmz a...,Nonsense,0,EAA,1,0,Valid,0,0,0,1,1,0,0
4,956,All aTScRSXcJu are XmnCyroGRq. All XmnCyroGRq ...,Nonsense,1,AAA,1,1,Valid,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,110,All DtQctcCOEa are LwpSUcgkmz. Some LwpSUcgkmz...,Nonsense,1,AII,1,1,Valid,0,0,1,0,0,0,1
1022,384,Some xwDELNqzrh are BBaAHDWjNs. Some BBaAHDWjN...,Nonsense,0,III,1,0,Invalid,1,0,0,0,0,1,1
1023,715,No iUyshCysyl are mRTPduzuwG. All mRTPduzuwG a...,Nonsense,1,EAE,1,1,Invalid,0,1,0,0,0,1,0
1024,767,All iUyshCysyl are mRTPduzuwG. Some mRTPduzuwG...,Nonsense,0,AIA,1,0,Invalid,1,0,0,0,0,1,0


In [ ]:
excel_df.to_excel("llama_results_nonsense.xlsx")